In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [26]:
cols = ['engine_id', 'cycle', 'op1', 'op2', 'op3'] + \
       [f'sensor_{i}' for i in range(1, 22)]

df = pd.read_csv(
    '../data/raw/train_FD001.txt',
    delim_whitespace=True,
    header=None,
    names=cols
)



C:\Users\Admin\AppData\Local\Temp\ipykernel_13244\1872373258.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


In [27]:
max_cycles = df.groupby('engine_id')['cycle'].max()

df['RUL'] = df.apply(
    lambda row: max_cycles[row['engine_id']] - row['cycle'],
    axis=1
)


In [28]:
drop_cols = ['op3', 'sensor_1', 'sensor_5', 'sensor_10']  # example
df_clean = df.drop(columns=drop_cols)


In [29]:
RUL_CAP = 130
df['RUL'] = df['RUL'].clip(upper=RUL_CAP)



In [30]:
window = 5

for col in [c for c in df_clean.columns if 'sensor' in c]:
    df_clean[f'{col}_rollmean'] = (
        df_clean.groupby('engine_id')[col]
        .rolling(window)
        .mean()
        .reset_index(level=0, drop=True)
    )


In [31]:
df_clean = df_clean.dropna()
